In [33]:
import sqlite3
import pandas as pd

import os
import requests
import json
import alpaca_trade_api as tradeapi
import codecs
import webbrowser
import ipywidgets as widgets
import hvplot.pandas

import warnings
warnings.filterwarnings('ignore')

In [34]:
from model import mvc_exceptions as mvc_exc

from model import db_backend as db_bkend

from dotenv import load_dotenv

#from sqlalchemy import create_engine, inspect

from IPython.display import display, clear_output

In [35]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [36]:
#import environment variables
load_dotenv()
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
type(alpaca_api_key)
    
    
#Create the Alpaca API object
alpaca = tradeapi.REST(
alpaca_api_key,
alpaca_secret_key,
api_version='v2')

In [37]:
### Image Widget

file = open("./resources/images/investment-firm.png", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='png',
                    #width='400',
                    height= '5')

label_headline = widgets.Label(
    #value='Welcome to Three Amigos Investment Firm',
    style={'description_width': 'initial'}

)
vbox_headline = widgets.VBox([image_headline, label_headline])

#display(vbox_headline)

In [38]:
text_0 = widgets.HTML(value="<h1>Welcome to Three Amigos Investment Firm</h1>")
vbox_text = widgets.VBox([text_0])

In [39]:
# widget for selecting the funds for review
fund_to_review = widgets.Dropdown(
    options=['ARKK', 'YOLO', 'MSOS', 'IVY'],
    value='ARKK',
    # description='Select the funds to review',
    disabled=False,
)

display(fund_to_review.value)

'ARKK'

In [40]:
holdings_symbol =  fund_to_review.value  #We'll just stick to using  'ARKK' for now until we add other funds
holdings_url = 'https://arkfunds.io/api/v2/etf/holdings' 

portfolio_holding_df = db_bkend.import_portfolio_holding(holdings_symbol, holdings_url)

display(portfolio_holding_df)
#display(portfolio_holding_df.tail())

,ticker,weight,company
0,TSLA,9.91,TESLA INC
1,TDOC,6.62,TELADOC HEALTH INC
2,U,6.29,UNITY SOFTWARE INC
3,COIN,6.27,COINBASE GLOBAL INC -CLASS A
4,ROKU,5.17,ROKU INC
5,ZM,5.00,ZOOM VIDEO COMMUNICATIONS-A
6,SPOT,4.14,SPOTIFY TECHNOLOGY SA
7,SHOP,4.11,SHOPIFY INC - CLASS A
8,SQ,3.59,SQUARE INC - A
9,TWLO,3.54,TWILIO INC - A


In [41]:
# Review portfolio holdings and store to db

# create a table for the funds using the dataframe of the funds imported
# use the holding_symbol as the table name
table_name = holdings_symbol

#use the memory database engine/ connection
conn = db_bkend.connect_to_db()

#assign the portfolio_holding_df to table_data_df
table_data_df = portfolio_holding_df

#create the table from the dataframe
db_bkend.create_table_from_dataframe(conn, table_name, table_data_df)



This is a new connection to in memory SQLite db...


In [42]:
# You can create a second table of a portfolio of a list of different assets if desire
table_name = 'Portfolio_01'

db_bkend.create_table(conn, table_name)

# update the table with the list of assets you want in the portfolio

weight = 12.0
company="ABC Inc"
ticker = "TICK"
db_bkend.insert_one(conn, ticker, weight, company, table_name)

In [43]:
 #Check if table was created in the db
table_names = conn.table_names()

display(table_names)
#inspector.get_table_names()
#db_bkend.select_all(conn, 'ARKK')

['ARKK', 'Portfolio01', 'sqlite_sequence']

In [44]:
#table_name = 'TABLE02'
#weight = 12.0
#company="ABC Inc"
#ticker = "TICK"
#db_bkend.insert_one(conn, ticker, weight, company, table_name)

#asset_ticker = ticker
#db_bkend.select_one(conn, asset_ticker, table_name)

In [45]:
db_bkend.update_one(conn, ticker, 15.0, company, table_name)

In [46]:
db_bkend.select_all(conn, table_name='Portfolio01')

[{'id': 1, 'ticker': 'TICK', 'weight': 15.0, 'company': 'ABC Inc'}]

In [47]:
#db_bkend.delete_one(conn, 'TICK', table_name='TABLE02')
#db_bkend.delete_one(conn, 'TICK', 'TABLE02')

In [48]:


#page = widgets.HBox([vbox_headline, vbox_headline])

web_page = widgets.VBox([vbox_headline, text_0 ])

display(web_page)

In [49]:
# For our purposes we want to focus on the 'ticker','weight', and 'company' columns of the dataframe.  This will allow us to perform historical research with the Alpaca API as well as plot the weights of the portfolio stocks.
initial_filtered_df = portfolio_holding_df[['ticker', 'weight', 'company']].sort_values(by = 'weight')


# Note that for our Monte Carlo simulations, we will need to divide the weights column by 100 since the sum of weights for the simulation needs to be 1, and the dataframe is configured for the sum to be 100.

initial_filtered_bar = initial_filtered_df.hvplot.bar(x='ticker', y = 'weight', hover_color = 'red', rot=90, title = 'Stock tickers and their corresponding weights in the portfolio')
display(initial_filtered_bar)
#display(initial_filtered_df)

:Bars   [ticker]   (weight)